<a href="https://colab.research.google.com/github/prudhvi193/Distributed-Market-Basket-Analysis-using-PySpark/blob/main/Market_Basket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Name : Prudhviraj Sheela
# OSU CWID : A20228857
!pip install pyspark

     |████████████████████████████████| 204.2MB 65kB/s 
     |████████████████████████████████| 204kB 45.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=62e63837e731e5256445e8023aeec7923a2725dba369b49c9606e8c33a10919a
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
import pyspark # Importing PySpark
from pyspark import SparkContext 
sc = SparkContext("local","Frequent Item Set and Association Rules Building") # Creating the Spark Context

In [ ]:
rdd1 = sc.textFile("/content/browsing.txt")

In [ ]:
from collections import Counter

s = 85 # Given Support Value
k = 2 # Initial Count for Frequent Items

def baskets(lines): # This function gives the baskets from the given input file and returns it to the RDD
    baskets = [[[item for item in line.strip().split(" ")] for line in lines]]
    return baskets
rdd2 = sc.union([rdd1.mapPartitions(baskets)]) # This rdd is used for obtaining the baskets from the given dataset
rdd2.collect()

[[['FRO11987', 'ELE17451', 'ELE89019', 'SNA90258', 'GRO99222'],
  ['GRO99222',
   'GRO12298',
   'FRO12685',
   'ELE91550',
   'SNA11465',
   'ELE26917',
   'ELE52966',
   'FRO90334',
   'SNA30755',
   'ELE17451',
   'FRO84225',
   'SNA80192'],
  ['ELE17451', 'GRO73461', 'DAI22896', 'SNA99873', 'FRO86643'],
  ['ELE17451', 'ELE37798', 'FRO86643', 'GRO56989', 'ELE23393', 'SNA11465'],
  ['ELE17451',
   'SNA69641',
   'FRO86643',
   'FRO78087',
   'SNA11465',
   'GRO39357',
   'ELE28573',
   'ELE11375',
   'DAI54444'],
  ['ELE17451',
   'GRO73461',
   'DAI22896',
   'SNA99873',
   'FRO18919',
   'DAI50921',
   'SNA80192',
   'GRO75578'],
  ['ELE17451',
   'ELE59935',
   'FRO18919',
   'ELE23393',
   'SNA80192',
   'SNA85662',
   'SNA91554',
   'DAI22177'],
  ['ELE17451',
   'SNA69641',
   'FRO18919',
   'SNA90258',
   'ELE28573',
   'ELE11375',
   'DAI14125',
   'FRO78087'],
  ['ELE17451',
   'GRO73461',
   'DAI22896',
   'SNA80192',
   'SNA85662',
   'SNA90258',
   'DAI46755',
   'FRO8117

In [ ]:
def filter_itemsets(itemset_dict,s): # This function filters the frequent itemsets which satisfy the support value
    # Filter frequent items
    for k in list(itemset_dict.keys()):
        # Remove non-frequent items
      if itemset_dict[k] < s:
        del itemset_dict[k] # If support does not satisfy then the element is deleted from the list
    return itemset_dict

def singletons(x,s): # This function is used to generate the singletons from the passed rdd 
    # Here x is the basket and s is the support given
    singletons = dict() 
    for b in x:
      for a in b:
        try:
          singletons[a] += 1 # If the singleton exists then its value gets incremented
        except KeyError as e:
          singletons[a] = 1 # Else its value remains as one
    singletons = filter_itemsets(singletons,s)
    return singletons,x # Returns the singletons and also the original baskets list inorder to withold it in the next rdd
    
rdd3 = rdd2.map(lambda x : singletons(x,s)) # This rdd helps in obtaining the singletons
#rdd3.collect()

In [ ]:
final_item = [] # This list is used to store all the tuples 
# The below function calculates the itemset generation for 2,3,4 sizes
def frequent_itemsets(prev,frequent_items,baskets,s,k):
  candidate_itemsets = dict() # Initializing the candidate_itemsets
  while True and k<=4: # Evaluated the loop until we obtain 4 pair frequent itemsets and then terminates
    if (k==2): # If the 'k' value is of 2-frequent items then this loop evaluates for its candidate items
      for key in prev:
        for item in frequent_items:
          union = (key,) + (item,) # Here key and value are of String data type
          union = tuple(sorted(union)) # Obtains the union of two items obtained from key, item
          if len(union) == k: # Checks if the len(union) satisfies k-frequent items length
            if all(value < 2 for value in dict(Counter(union)).values()): # Checks if there are elements repeated more than once and removes them as there must not be duplicates in the list
              candidate_itemsets[union] = 0
      for b in baskets: # For the below iteration I had calculated how many times the candidate itemset exists in the basket
        for j in candidate_itemsets:
          if list(j)<=b:
            candidate_itemsets[j] += 1
      candidate_itemsets = filter_itemsets(candidate_itemsets,s) # Here the filter function is called to remove the itemsets that are less than the support value
      final_item.append(prev) # In my final_item list I am restoring my singletons in order to traverse it to the next rdd to evaluate association rules
      final_item.append(candidate_itemsets) # Here the candidate itemsets for 2-frequent pairs is obtained
      k += 1 # Incrementing the k-value for next iteration
      frequent_itemsets(candidate_itemsets,frequent_items,baskets,s,k) # Calling the function to obtain next set of frequent items
    else:
      for key in prev:
        for item in frequent_items:
          union = key + (item,) # Here key and value are of Tuple and String Data Type, so I had converted it entirely to a string
          union = tuple(sorted(union)) # Obtains the union of two items obtained from key, item
          if len(union) == k:  # Checks if the len(union) satisfies k-frequent items length
            if all(value < 2 for value in dict(Counter(union)).values()): # Checks if there are elements repeated more than once and removes them as there must not be duplicates in the list
              candidate_itemsets[union] = 0
      for b in baskets: # For the below iteration I had calculated how many times the candidate itemset exists in the basket
        for j in candidate_itemsets:
          if list(j)<=b:
            candidate_itemsets[j] += 1
      candidate_itemsets = filter_itemsets(candidate_itemsets,s) # Here the filter function is called to remove the itemsets that are less than the support value
      final_item.append(candidate_itemsets) # Here the candidate itemsets for 2-frequent pairs is obtained
      k+=1 # Incrementing the k-value for next iteration
      frequent_itemsets(candidate_itemsets,frequent_items,baskets,s,k) # Calling the function to obtain next set of frequent items
    return final_item

rdd4 = rdd3.map(lambda x : frequent_itemsets(x[0],list(x[0].keys()),x[1],s,k))
#rdd4.collect()

In [ ]:
c = 90 # Given Confidence Threshold is 90%

from itertools import combinations # This package is used to obtain all kinds of combinations for the obtained frequent items

def get_support(frequent_itemsets,itemset): # This function returns the denominator support value for the associated itemset
  l = frequent_itemsets[len(itemset)-1] # It checks for the itemsets that are less by one of its length and evalueates it
  if len(itemset) == 1: # If len(itemset) is '1' then it is stored as string in a list, so I had used this to return value for singletons
    for key,value in l.items():
      if key == itemset[0]: # If the itemset exist in the singleton it returns its value which is the 'support_a'
        return value
  else: # If len(itemset) is more than '1' then it is stored as tuple in a list, so I had used this to return value for duplets, triplets and quadraples
    for key,value in l.items():
      itemset = tuple(sorted(itemset)) # This sorts the obtained itemset so that it can match to the key elements and return its associated support
      if key == itemset: # If the itemset exist in the (duplet) or (triplet) or (quadraple) it returns its value which is the 'support_a
        return value

def generate_association_rules(frequent_itemset,c): # This function is called through rdd which gives the association rules
  # In the above parameters 'frequent_itemsets' contains all the singletons, duplets, triplets and quadraples as a list & c is the confidence given
  assoc_rules = [] # All the association rules obtained are stored in this list
  length = len(frequent_itemset)
  for f in range(1,length): # We start iterating from the duplets then triplets and then quadraples
    for itemset,support in frequent_itemset[f].items(): # Now we iterate through itemset,support values to do our further computation in finding association rules
      length_itemset = len(itemset)
      for i in range(1,length_itemset): # In this iteration we try to obtain all the combinations that are possible using the itemset
        subsets = list(combinations(set(itemset), i))
        subsets1 = []
        for a in subsets: # In the below iteration I am converting all the tuples in my subset to list so that while calling the get_support function I can iterate it accordingly
          subsets1.append(list(a))
        for a in subsets1: # Once the subsets list is obtained we find the support values and confidence for the each itemset
          support_a = get_support(frequent_itemset,a) # Calling the support function to get the support for subsets
          if support_a is None: # If there exists not support value for element in subset it returns None, so we do not do anything and continue
            continue
          confidence = round((support * 100/ support_a),2) # Calculating confidence and rounding the obtained confidence value to 2-digits 
          if len(set(itemset).difference(set(a))) < 2: # This condition eliminates if there are more than 1 element on the right hand side of the association rule
            if confidence >= c : # If confidence opbtained is greater than given confidence value then we send them to association rules
              string1 = 'ID' + str(set(a)) + ' -> ' + 'ID' + str(set(itemset).difference(set(a))) + ';' + 'Confidence='
              string2 = "%"
              final_tuple = (string1,str(confidence),string2)
              assoc_rules.append(final_tuple)
              assoc_rules.sort(key = lambda x: float(x[1]),reverse = True) # Sorting the association rules in descending order based on its valuec
  return assoc_rules


rdd5 = rdd4.map(lambda x : generate_association_rules(x,c)) # The association rules are generated using this rdd
rdd6 = rdd5.map(lambda x : list(map("".join, x))) # Joining all the strings in a tuple which becomes a string
rdd7 = rdd6.flatMap(lambda x : x) # Combining all the strings in the list into one list


In [ ]:
rdd7.saveAsTextFile("/content/sample_data/Output.txt") # The output with association rules are saved in this specified text file